# LeNet5(5层参数可训练)
![](https://picgogogo.oss-cn-hangzhou.aliyuncs.com/img/LetNet5.png)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets.mnist as mnist
import tensorflow.keras.layers as layers
import numpy as np

# 优化器
from tensorflow.keras.optimizers import SGD

# 数据集

In [ ]:
### 准备数据 

# (((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

x_train = np.array(x_train, dtype=np.float32)
x_test = np.array(x_test, dtype=np.float32)

# 将数字转换成one-hot向量
def num_to_onehot(list):
    y = np.zeros(shape=(list.shape[0], 10))
    for i in range(list.shape[0]):
        y[i, list[i]] = 1
    return y

y_train = num_to_onehot(y_train)  
y_test = num_to_onehot(y_test)

# LeNet5

In [ ]:
### 定义神经网络

# 1. 层定义
# 输入层
x = keras.Input(shape=(28, 28, 1), dtype=tf.float32)  # shape是图片的shape， 与数量无关

# 卷积层
con1 = layers.Conv2D(filters=6,
                    kernel_size=(5, 5),
                    strides=(1,1),
                    padding="same",
                    activation="relu",
                    dtype=tf.float32)

# strides默认为pool_size
pool1 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")

con2 = layers.Conv2D(filters=16,
                    kernel_size=(5, 5),
                    strides=(1,1),
                    padding="valid",
                    activation="relu",
                    dtype=tf.float32)

pool2 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")
pool2_flatten = layers.Flatten()

# 全连接层
fc1 = layers.Dense(units=120, activation='relu')
fc2 = layers.Dense(units=84, activation='relu')
fc3 = layers.Dense(units=10, activation='softmax')

# 2. 定义模型
model = keras.Sequential(layers=[x, con1, pool1, con2, pool2, pool2_flatten, fc1, fc2, fc3])

# 3. 定义训练参数
# sgd = SGD(learning_rate=0.01)

## 在1.12中是lr
sgd = SGD(lr=0.01)

model.compile(
    optimizer=sgd,                      # 指定优化器
    loss='categorical_crossentropy',    # 指定损失函数
    metrics=['accuracy']
)

# 训练
model.fit(x_train, y_train, epochs=2)

# 最终结果
pred = model.predict(x_test)
accuracy = np.mean(np.argmax(pred, axis=1) == np.argmax(y_test, axis=1))
loss = model.evaluate(x_test, y_test)

print(accuracy, loss)

# 简单的封装

In [ ]:
class LetNet5:
    
    def __num_to_onehot(self, list):
        '''
        将数字转换成one-hot向量
        '''
        y = np.zeros(shape=(list.shape[0], 10))
        for i in range(list.shape[0]):
            y[i, list[i]] = 1
        return y
    
    def __init__(self, epoch):
        self.epoch = epoch
        self.get_model()
    
    def get_model(self):
        # 输入层
        self.x = keras.Input(shape=(28, 28, 1), dtype=tf.float32)  # shape是图片的shape， 与数量无关
#         self.x = layers.Input(shape=(28, 28, 1), dtype=tf.float32)

        # 卷积层
        self.con1 = layers.Conv2D(filters=6,
                            kernel_size=(5, 5),
                            strides=(1,1),
                            padding="same",
                            activation="relu",
                            dtype=tf.float32)

        # strides默认为pool_size
        self.pool1 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")

        self.con2 = layers.Conv2D(filters=16,
                            kernel_size=(5, 5),
                            strides=(1,1),
                            padding="valid",
                            activation="relu",
                            dtype=tf.float32)

        self.pool2 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")
        self.pool2_flatten = layers.Flatten()

        # 全连接层
        self.fc1 = layers.Dense(units=120, activation='relu')
        self.fc2 = layers.Dense(units=84, activation='relu')
        self.fc3 = layers.Dense(units=10, activation='softmax')
        
        self.model = keras.Sequential(layers=[self.x, self.con1, self.pool1, self.con2, self.pool2, self.pool2_flatten, self.fc1, self.fc2, self.fc3])
        
        sgd = SGD(0.01)

        self.model.compile(
            optimizer=sgd,                      # 指定优化器
            loss='categorical_crossentropy',    # 指定损失函数
            metrics=['accuracy']
        )
        
    def fit(self, x, y):
        return self.model.fit(x, self.__num_to_onehot(y), epochs=self.epoch)
    
    def evaluate(self, x_test, y_test):
        y_test = self.__num_to_onehot(y_test)
        
        pred = self.model.predict(x_test)
        accuracy = np.mean(np.argmax(pred, axis=1) == np.argmax(y_test, axis=1))
        return accuracy

In [ ]:
# 测试
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

x_train = np.array(x_train, dtype=np.float32)
x_test = np.array(x_test, dtype=np.float32)

net = LetNet5(50)
net.fit(x_train, y_train)
net.evaluate(x_test, y_test)